# 最小二乗法をやってみよう！

このページは以下のリンクより， google colaboratoryから動作させることができる．
- [Open with Colab](https://colab.research.google.com/github/crotsu/Bousai_AI/blob/master/chap2_Python/chap2_3_LSM.ipynb)

ここでは，PythonとライブラリのNumpyに慣れるために，最小二乗法を実装してみることにする．

## 目次
1. 真のモデルからデータを生成する
1. 真のモデルにノイズ（正規乱数）の影響を与えて生成する
1. 最小二乗法を適用する(次数1の近似関数を愚直に求める)
1. 最小二乗法を適用する(Numpyライブラリで一発で求める)
1. デモ（次数可変版）

## 1. 真のモデルからデータを生成する

真のモデルとして，以下の関数を仮定する．

$$
f(x) = x^4-8x^3+18x^2-3x+1
$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 真のモデル
def mathfunc(x):
    y = x**4 - 8*x**3 + 18*x**2 - 3*x + 1
    return y

# 定義域（xの範囲: -1.0から4.5）
# 生成するデータの個数は100個
xmin = -1.0
xmax = 4.5
num = 100

# xを連番で生成する．
dataX = np.linspace(xmin, xmax, num)

# ｘからｙを生成する．
dataY = mathfunc(dataX)

# グラフ表示
plt.scatter(dataX, dataY)
plt.xlim(xmin, xmax)
plt.grid(True)
plt.show()

## 2. データは，真のモデルにノイズ（正規乱数）の影響を受ける

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 真のモデル
def mathfunc(x):
    y = x**4 - 8*x**3 + 18*x**2 - 3*x + 1
    return y

# 定義域（xの範囲: -1.0から4.5）
# 生成するデータの個数は100個
xmin = -1.0
xmax = 4.5
num = 100

# 乱数の種を設定
np.random.seed(0)

# ランダムにxを生成して，昇順にソート
dataX = (xmax - xmin) * np.random.rand(num) + xmin
dataX = np.sort(dataX)

# xからｙを生成する
dataY = mathfunc(dataX)

# yにノイズ（正規乱数（平均0．0， 標準偏差2.0））を加える
dataY += np.random.normal(loc=0.0, scale=2.0, size=num)

# グラフ表示
plt.scatter(dataX, dataY)
plt.xlim(xmin, xmax)
plt.grid(True)
plt.show()

In [ ]:
# データの確認 (x)
dataX

In [ ]:
# データの確認 (y)
dataY

## 3. 最小二乗法を適用する(次数1の近似関数を愚直に求める)
次数1，つまりy=ax+bの直線で近似する．

In [ ]:
# 連立1次方程式 Ax = bの行列Aとベクトルbを求める
# 次数1のとき，正規方程式の行列は2行2列になる
matrixA = np.zeros((2,2))
b = np.zeros((2,1))

In [ ]:
matrixA[0,0] = sum(dataX * dataX)
matrixA[0,1] = sum(dataX)
matrixA[1,0] = sum(dataX)
matrixA[1,1] = len(dataX)

b[0,0] = sum(dataX*dataY)
b[1,0] = sum(dataY)

In [ ]:
# データの確認
matrixA

In [ ]:
# データの確認
b

In [ ]:
# 連立1次方程式 Ax = bを解く
#
# 公式を使って逆行列を求めて，解を求める

inv = 1/(matrixA[0,0]*matrixA[1,1] - matrixA[0,1]*matrixA[1,0])*np.array([[matrixA[1,1],-1*matrixA[1,0]],[-1*matrixA[1,0],matrixA[0,0]]])
ans = np.dot(inv, b)
print(ans)

In [ ]:
# 連立1次方程式 Ax = bを解く
#
# Numpyで逆行列を求めて，解を求める

inv = np.linalg.inv(matrixA)
ans = np.dot(inv, b)
print(ans)

In [ ]:
# 連立1次方程式 Ax = bを解く
#
# Numpyで連立1次方程式を解く
# 数値計算的には，この方法がベスト
# 解が求まるなら逆行列を求める必要はないから

ans = np.linalg.solve(matrixA, b)
print(ans)

In [ ]:
# 求めたパラメータをグラフに表示する

# 求めた関数
def solve_func(x):
    y = ans[0] * x + ans[1]
    return y

# 定義域（xの範囲: -1.0から4.5）
# 生成するデータの個数は100個
xmin = -1.0
xmax = 4.5
num = 100 

# xを生成
# xminからxmaxまでをnum個で区切る
x = np.linspace(xmin, xmax, num)

# ｘからｙを生成する．
y = solve_func(x)

# グラフ表示
plt.plot(x, y, color='red') # 求めたパラメータから直線を引く

plt.scatter(dataX, dataY) # 散布図
plt.xlim(xmin, xmax)
plt.grid(True)
plt.show()

## 4. 最小二乗法を適用する(Numpyライブラリで一発で求める)
実はライブラリを使えば，もっと簡単に求めることができる．

In [ ]:
# numpyのライブラリを使ってパラメータを求める(次数1)
ans = np.polyfit(dataX, dataY, 1)
print(ans)

In [ ]:
# numpyのライブラリを使ってパラメータを求める(次数4)
ans = np.polyfit(dataX, dataY, 4)
print(ans)

次数4に対応させるために関数を修正する必要がある

In [ ]:
# 求めたパラメータをグラフに表示する

# 求めた関数
def solve_func(x):
    y = ans[0]*x*x*x*x + ans[1]*x*x*x + ans[2]*x*x + ans[3]*x + ans[4]
    return y

# 定義域（xの範囲: -1.0から4.5）
# 生成するデータの個数は100個
xmin = -1.0
xmax = 4.5
num = 100 

# xを生成
# xminからxmaxまでをnum個で区切る
x = np.linspace(xmin, xmax, num)

# ｘからｙを生成する．
y = solve_func(x)

# グラフ表示
plt.plot(x, y, color='red')

plt.scatter(dataX, dataY) # 散布図
plt.xlim(xmin, xmax)
plt.grid(True)
plt.show()

## 5. デモ（次数可変版）
次数が増減してもそれに応じて関数を修正する必要がないプログラムを示す．
データ生成と最小二乗法のプログラムを以下にまとめるので，パラメータをいろいろ変更して試してみると良い．

### データ生成
データの個数，乱数の種，標準偏差を変更して様々なデータを生成してみる．

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 真のモデル
def mathfunc(x):
    y = x**4 - 8*x**3 + 18*x**2 - 3*x + 1
    return y

# 定義域（xの範囲: -1.0から4.5）
# 生成するデータの個数は100個
xmin = -1.0
xmax = 4.5
num = 100

# 乱数の種を設定
np.random.seed(0)

# ランダムにxを生成して，昇順にソート
dataX = (xmax - xmin) * np.random.rand(num) + xmin
dataX = np.sort(dataX)

# xからｙを生成する
dataY = mathfunc(dataX)

# yにノイズ（正規乱数（平均0．0， 標準偏差2.0））を加える
dataY += np.random.normal(loc=0.0, scale=2.0, size=num)

# グラフ表示
plt.scatter(dataX, dataY) # 散布図
plt.xlim(xmin, xmax)
plt.grid(True)
plt.show()

### 最小二乗法
次数を変更してみる．

次数が多くなるほど，過学習していることがわかる．

In [ ]:
# 次数
dim = 4

# 最小二乗法でパラメータを求める．
ans = np.polyfit(dataX, dataY, dim)


# 求めたパラメータをグラフに表示する

# 定義域（xの範囲: -1.0から4.5）
# 生成するデータの個数は100個
xmin = -1.0
xmax = 4.5
num = 100 

# xを生成
# xminからxmaxまでをnum個で区切る
x = np.linspace(xmin, xmax, num)

# ｘからｙを生成する．
y = solve_func(x)

# グラフ表示
# 求めたパラメータから，それを係数とする多項式を作る．
plt.plot(dataX, np.poly1d(ans)(dataX),color='red')

plt.scatter(dataX, dataY)
plt.xlim(xmin, xmax)
plt.grid(True)
plt.show()